### Scenario 1: A single data scientist participating in an ML competition

MLflow setup:

 - Tracking server: no
 - Backend store: local filesystem
 - Artifacts store: local filesystem

 
The experiments can be explored locally by launching the MLflow UI.

In [6]:

import mlflow

In [7]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [8]:
print(mlflow.__version__)

3.7.0


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1765120390682, experiment_id='1', last_update_time=1765120390682, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1765120032098, experiment_id='0', last_update_time=1765120032098, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]


### Creating an experiment and logging a new run

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models", input_example=X[0:1])

    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/12/09 14:45:36 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/12/09 14:45:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


default artifacts URI: 'mlflow-artifacts:/2/352a98ef2c394e8c92f22de41cf4a10b/artifacts'
🏃 View run unleashed-dove-270 at: http://127.0.0.1:5000/#/experiments/2/runs/352a98ef2c394e8c92f22de41cf4a10b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [11]:

mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1765287936222, experiment_id='2', last_update_time=1765287936222, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1765120390682, experiment_id='1', last_update_time=1765120390682, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1765120032098, experiment_id='0', last_update_time=1765120032098, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

In [12]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


### Interacting with the model registry


In [14]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [15]:

from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")